In [114]:
import statsmodels.api as sm
import numpy as np
from econml.dml import LinearDML
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from xgboost import XGBClassifier, XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.neural_network import MLPClassifier, MLPRegressor
from catboost import CatBoostRegressor, CatBoostClassifier
from scipy.stats import logistic

MC_N = 1
MC_θ = np.zeros((MC_N,4))
MC_y = np.zeros((MC_N,4))
MC_t = np.zeros((MC_N,4))

for j in range(MC_N):
    import wooldridge
    df = wooldridge.data('jtrain2')
    #df['avg'] = 0.5 * (df.re74+df.re75)
    df = df.dropna()
    #df = df[df.avg <= 15]
    y = df.re78
    t = df.train
    x = df[['age', 'educ', 'black', 'hisp', 'married', 're74', 're75']]
    
    # OLS - Full Estimation
    model_OLS = sm.OLS(y, np.c_[t,x])
    res = model_OLS.fit()
    θ_OLS = res.params[0]
    # OLS First Stage: Y
    model_OLS = sm.OLS(y, np.c_[x])
    res_y = model_OLS.fit()
    θ_OLS_y = res_y.params[0]    
    # Logistic First Stage
    clf = LogisticRegression(random_state=0).fit(x, t)
    
    # DML Lasso
    model_Lasso = LinearDML(discrete_treatment=True, random_state=1, cv=1)
    model_Lasso.fit(y, t, X=None,W=x)
    θ_DMLL = model_Lasso.intercept_

    # DML RF
    model_XGB = LinearDML(discrete_treatment=True, cv=1,
                          model_y = CatBoostRegressor(learning_rate = 1, max_depth = 8,verbose = False), 
                          model_t = CatBoostClassifier(learning_rate = 1, max_depth = 8,verbose = False))
    model_XGB.fit(y.ravel(), t.ravel(), X=None,W=x)
    θ_DMLRF = model_XGB.intercept_
    
    # DML NN - First Stage
    model_NN = LinearDML(discrete_treatment=True, cv =1,
                         model_y = MLPRegressor(random_state=1,
                                                 hidden_layer_sizes=(500,100,50), 
                                                 batch_size = x.shape[0],
                                                 momentum = 0.95, 
                                                 max_iter=50000, 
                                                 learning_rate_init=0.01, 
                                                 verbose=True), 
                         model_t = MLPClassifier(random_state=1,
                                                 hidden_layer_sizes=(500,100,50), 
                                                 batch_size = x.shape[0],
                                                 momentum = 0.95, 
                                                 max_iter=50000, 
                                                 learning_rate_init=0.01, 
                                                 verbose=True))
    model_NN.fit(y.ravel(), t.ravel(), X=None,W=x)
    θ_DMLRF = model_NN.intercept_


    MC_θ[j] = [θ_OLS, model_Lasso.intercept_, model_XGB.intercept_, model_NN.intercept_]
    MC_y[j] = [res_y.rsquared, np.mean(model_Lasso.nuisance_scores_y), np.mean(model_XGB.nuisance_scores_y),np.mean(model_NN.nuisance_scores_y)]
    MC_t[j] = [clf.score(x,t), np.mean(model_Lasso.nuisance_scores_t), np.mean(model_XGB.nuisance_scores_t),np.mean(model_NN.nuisance_scores_t)]
    
    

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


Iteration 1, loss = 0.87673972
Iteration 2, loss = 2.81772548
Iteration 3, loss = 0.97819756
Iteration 4, loss = 1.46328709
Iteration 5, loss = 1.00752766
Iteration 6, loss = 0.68627108
Iteration 7, loss = 0.86141401
Iteration 8, loss = 0.71117988
Iteration 9, loss = 0.70135705
Iteration 10, loss = 0.76582677
Iteration 11, loss = 0.73543689
Iteration 12, loss = 0.68300651
Iteration 13, loss = 0.69268679
Iteration 14, loss = 0.72514142
Iteration 15, loss = 0.71792968
Iteration 16, loss = 0.68882130
Iteration 17, loss = 0.67749800
Iteration 18, loss = 0.68885446
Iteration 19, loss = 0.69874122
Iteration 20, loss = 0.69413188
Iteration 21, loss = 0.68171368
Iteration 22, loss = 0.67610814
Iteration 23, loss = 0.67774236
Iteration 24, loss = 0.68188685
Iteration 25, loss = 0.68231328
Iteration 26, loss = 0.67852480
Iteration 27, loss = 0.67455167
Iteration 28, loss = 0.67331370
Iteration 29, loss = 0.67477121
Iteration 30, loss = 0.67649672
Iteration 31, loss = 0.67641246
Iteration 32, los

Iteration 17, loss = 21.51082832
Iteration 18, loss = 21.52714540
Iteration 19, loss = 21.41552616
Iteration 20, loss = 21.39469498
Iteration 21, loss = 21.37873718
Iteration 22, loss = 21.28546719
Iteration 23, loss = 21.27521163
Iteration 24, loss = 21.20324764
Iteration 25, loss = 21.17660658
Iteration 26, loss = 21.14461116
Iteration 27, loss = 21.09134314
Iteration 28, loss = 21.09030803
Iteration 29, loss = 21.04756538
Iteration 30, loss = 21.04815424
Iteration 31, loss = 20.99662872
Iteration 32, loss = 20.98980611
Iteration 33, loss = 20.95286921
Iteration 34, loss = 20.92852209
Iteration 35, loss = 20.88354573
Iteration 36, loss = 20.86430783
Iteration 37, loss = 20.82304569
Iteration 38, loss = 20.79964854
Iteration 39, loss = 20.76189164
Iteration 40, loss = 20.73153170
Iteration 41, loss = 20.68861193
Iteration 42, loss = 20.65733135
Iteration 43, loss = 20.62155880
Iteration 44, loss = 20.56941518
Iteration 45, loss = 20.53284167
Iteration 46, loss = 20.48791134
Iteration 

In [112]:
from prettytable import PrettyTable
from PIL import Image, ImageDraw, ImageFont
table = PrettyTable()
table.field_names = ['Var', 'OLS','DML-L','DML-Boost','DML-NN']
a = ['θ_hat']+ np.mean(MC_θ, axis = 0).tolist()
table.add_row(a)
a = ['First Stage Y R2']+ np.mean(MC_y, axis = 0).tolist()
table.add_row(a)
a = ['First Stage D Accuracy']+ np.mean(MC_t, axis = 0).tolist()
table.add_row(a)
table.float_format = '0.3'
print(table)

+------------------------+-------+--------+-----------+--------+
|          Var           |  OLS  | DML-L  | DML-Boost | DML-NN |
+------------------------+-------+--------+-----------+--------+
|         θ_hat          | 1.694 | 1.765  |   1.393   | 1.830  |
|    First Stage Y R2    | 0.414 | -0.007 |   0.827   | 0.189  |
| First Stage D Accuracy | 0.589 | 0.582  |   0.894   | 0.694  |
+------------------------+-------+--------+-----------+--------+
